Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [31]:
# Other Imports
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import cross_val_score, cross_val_predict,train_test_split
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [3]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

## **Set Up:**

### **Wrangle Train/Test the same way**

In [4]:
def wrangler(X): 
  # prevent overwriting original
  X = X.copy()
  
  # replace outlier with zero
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # drop duplicates and futiles
  duplicates = ['extraction_type','extraction_type_group','payment',
                'source_type', 'quality_group', 'quantity_group',
                'waterpoint_type_group','management']
  nugatory   = ['recorded_by','scheme_name']

  X = X.drop(columns= duplicates)
  X = X.drop(columns= nugatory)

  # Creating nans to be imputed later
  cols_with_zeros = ['longitude', 'latitude', 'population',
                      'gps_height', 'construction_year']
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)  
  
  # Reduce cardinality of 'funder' feature 
  Top20funder = X['funder'].value_counts(normalize=True)[:20]
  X.loc[~X['funder'].isin(Top20funder), 'funder'] = 'Other'

  # And 'installer' feature
  Top20install = X['installer'].value_counts(normalize=True)[:20]
  X.loc[~X['installer'].isin(Top20install),'installer'] = 'Other'

  # Changing region and district code features to categorical features
  X['region_code'] = X['region_code'].astype(str)
  X['district_code'] = X['district_code'].astype(str)

  # Changing date recorded to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], 
                                      infer_datetime_format=True)
  
  # Extracting year data from date recorded and dropping superfluous feature
  X['year_recorded'] = X['date_recorded'].dt.year
  X = X.drop(columns='date_recorded')

  # Engineering Age and Age_missing feature
  X['age'] = X['year_recorded'] - X['construction_year']
  X['age_MISSING'] = X['age'].isnull()

  return X

In [5]:
Train = wrangler(train)
Test  = wrangler(test)

### **Select Features and Arrange X and Y:**

In [6]:
# Target is status_group
target = 'status_group'

# Drop columns intuitively deemed unnecessary
super = [target,'id','wpt_name','num_private','region_code','lga',
         'ward','public_meeting','payment_type']

# Remaining dataframe
train_features = Train.drop(columns=super)

# Get a list of the numeric features
numeric = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 100
categorical = cardinality[cardinality <= 100].index.tolist()

# Combine those lists 
features = numeric + categorical

In [7]:
# Arrange X and Y 
x_train = Train[features]
x_test  = Test[features]

y_train = Train[target]

## **Cross_val_score with Random Forest Classifier:**

In [10]:
# make pipeline 
pipeline = make_pipeline(ce.OrdinalEncoder(),
                         IterativeImputer(max_iter=10,
                                          initial_strategy='mean',
                                          imputation_order='random', 
                                          random_state=24),
                         RandomForestClassifier(n_estimators= 300,
                                                min_samples_leaf= 2,
                                                min_samples_split= 3, 
                                                n_jobs=-1, 
                                                random_state=24))
# select number of folds and apply cross_val_score
k = 5
scores = cross_val_score(pipeline, x_train, y_train, cv=k)
# print results
print(f'Mean Absolute Error for {k} folds:',scores)

Mean Absolute Error for 5 folds: [0.81161616 0.80589226 0.81321549 0.80892256 0.80824916]


In [11]:
# Mean of the folds
scores.mean()

0.8095791245791245

## **Hyperparameters:**

In [35]:
# Create blank pipeline
conveyor = make_pipeline(ce.OrdinalEncoder(),
                         IterativeImputer(random_state=24),
                         RandomForestClassifier(random_state=24))
# create dictionary for parameter distributions
params = {'iterativeimputer__max_iter': [5,10,15,None],
          'iterativeimputer__initial_strategy': ['mean','most_frequent','median'],
          'iterativeimputer__imputation_order': ['ascending','descending','random'],
          'randomforestclassifier__n_estimators': randint(50,400),
          'randomforestclassifier__min_samples_leaf': [3,4,5,None],
          'randomforestclassifier__min_samples_split':[3,4,6,None],
          'randomforestclassifier__max_depth': [5,10,20,None],
          'randomforestclassifier__max_features': uniform(0,1)}
# create randomized search with params
search = RandomizedSearchCV(conveyor,
                            param_distributions=params,
                            n_iter=3,
                            cv=4,
                            scoring='accuracy',
                            verbose=10,
                            n_jobs=-1,
                            return_train_score=True)


In [36]:
# Fit search to Train data
search.fit(x_train,y_train);

Fitting 4 folds for each of 3 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  2.4min finished


In [37]:
# Print results
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', search.best_score_)

Best hyperparameters {'iterativeimputer__imputation_order': 'descending', 'iterativeimputer__initial_strategy': 'most_frequent', 'iterativeimputer__max_iter': 15, 'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.29484997290394865, 'randomforestclassifier__min_samples_leaf': 4, 'randomforestclassifier__min_samples_split': 3, 'randomforestclassifier__n_estimators': 220}
Cross-validation MAE 0.8044107744107745


### **Prediction:**

In [38]:
Pipeline = search.best_estimator_
Pipeline.score(x_train,y_train)

0.8871548821548821

In [39]:
y_pred = Pipeline.predict(x_test)
y_pred

array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'non functional'], dtype=object)

In [40]:
# Submit to Kaggle
submission = test[['id']].copy()
submission['status_group'] = y_pred
submission.to_csv('Meghan_Felker_TWPP4.csv', index=False)